In [35]:
from pyspark.sql import SparkSession

In [36]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

In [37]:
spark=SparkSession.builder.appName('parking_violations').getOrCreate()

In [38]:
parking_violdf=spark.read.format("csv").option('inferSchema','True').option('header','true').\
load('Parking_Violations_Issued_-_Fiscal_Year_2017.csv')

In [39]:
parking_violdf.schema

StructType(List(StructField(Summons Number,LongType,true),StructField(Plate ID,StringType,true),StructField(Registration State,StringType,true),StructField(Plate Type,StringType,true),StructField(Issue Date,StringType,true),StructField(Violation Code,IntegerType,true),StructField(Vehicle Body Type,StringType,true),StructField(Vehicle Make,StringType,true),StructField(Issuing Agency,StringType,true),StructField(Street Code1,IntegerType,true),StructField(Street Code2,IntegerType,true),StructField(Street Code3,IntegerType,true),StructField(Vehicle Expiration Date,IntegerType,true),StructField(Violation Location,IntegerType,true),StructField(Violation Precinct,IntegerType,true),StructField(Issuer Precinct,IntegerType,true),StructField(Issuer Code,IntegerType,true),StructField(Issuer Command,StringType,true),StructField(Issuer Squad,StringType,true),StructField(Violation Time,StringType,true),StructField(Time First Observed,StringType,true),StructField(Violation County,StringType,true),Stru

In [40]:
parking_violdf.limit(5).toPandas()
# parking_violdf.show(5).toPandas()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,None,0,None,FAILURE TO STOP AT RED LIGHT,None,None,None
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,None,0,None,FAILURE TO STOP AT RED LIGHT,None,None,None
2,4631633384,AVM7975,NY,PAS,03/09/2017,36,SUBN,GMC,V,0,...,GY,NaN,2010,None,0,None,PHTO SCHOOL ZN SPEED VIOLATION,None,None,None
3,8196557280,GWB7054,NY,PAS,01/18/2017,70,SUBN,TOYOT,T,59590,...,BL,NaN,2015,None,0,05,70A-Reg. Sticker Expired (NYS),None,None,None
4,4631184358,EXZ9820,NY,PAS,03/02/2017,36,4DSD,HONDA,V,0,...,GR,NaN,1997,None,0,None,PHTO SCHOOL ZN SPEED VIOLATION,None,None,None


In [41]:
parking_violation_2017=parking_violdf.filter(col('Issue Date').contains('2017')).\
select(col('Summons Number').alias('ticket'),\
       col('Registration State').alias('registrations_state'),\
       col('Issue Date').alias('issue_date'),\
       col('Violation Code').alias('violation_code'),\
      col('Vehicle Body Type').alias('vehicle_body_type'),\
      col('Vehicle Make').alias('vehicle_make'),
      col('Street Code1').alias('street_code1'),\
      col('Street Code2').alias('street_code2'),\
      col('Street Code3').alias('street_code3'),\
      col('Violation Precinct').alias('violation_precinct'),\
      col('Issuer Precinct').alias('issuer_precinct'),\
      col('Violation Time').alias('violation_time'))

In [42]:
parking_violation_2017.count()

5431918

In [43]:
parking_violation_2017.select('ticket').alias('count').distinct().count()

5431918

In [44]:
parking_violation_2017.select('registrations_state').distinct().count()

65

In [45]:
parking_violation_2017.filter('street_code1=0 or street_code2=0 or street_code3=0').select('ticket').count()

1816814

In [46]:
parking_violation_2017.groupby(col('violation_code')).count().withColumnRenamed('count','violation_code_freq').\
sort(col('violation_code_freq').desc()).\
select('violation_code','violation_code_freq').limit(5).toPandas()

,violation_code,violation_code_freq
0,21,768087
1,36,662765
2,38,542079
3,14,476664
4,20,319646


In [47]:
parking_violation_2017.groupby(col('vehicle_body_type')).count().withColumnRenamed("count","vehicle_body_type_freq").\
sort(col('vehicle_body_type_freq').desc()).select('vehicle_body_type','vehicle_body_type_freq').\
toPandas().head(5)

,vehicle_body_type,vehicle_body_type_freq
0,SUBN,1883954
1,4DSD,1547312
2,VAN,724029
3,DELV,358984
4,SDN,194197


In [48]:
parking_violation_2017.groupby(col('vehicle_make')).count().withColumnRenamed("count","vehicle_make_freq").\
sort(col('vehicle_make_freq').desc()).select('vehicle_make','vehicle_make_freq').\
toPandas().head(5)

,vehicle_make,vehicle_make_freq
0,FORD,636844
1,TOYOT,605291
2,HONDA,538884
3,NISSA,462017
4,CHEVR,356032


In [49]:
parking_violation_2017.filter('violation_precinct !=0').groupby(col('violation_precinct')).count().withColumnRenamed("count","violation_freq").\
sort(col('violation_freq').desc()).select('violation_precinct','violation_freq').\
toPandas().head(5)

,violation_precinct,violation_freq
0,19,274445
1,14,203553
2,1,174702
3,18,169131
4,114,147444


In [50]:
parking_violation_2017.filter('issuer_precinct !=0').groupby(col('issuer_precinct')).\
count().withColumnRenamed("count","violation_freq").\
sort(col('violation_freq').desc()).select('issuer_precinct','violation_freq').\
toPandas().head(5)

,issuer_precinct,violation_freq
0,19,266961
1,14,200495
2,1,168740
3,18,162994
4,114,144054


In [51]:
precinct_viol_count=parking_violation_2017.filter('issuer_precinct !=0').\
groupBy('issuer_precinct','violation_code').\
count().sort(col('count').desc())

In [52]:
precinct_viol_count.show(5)

+---------------+--------------+-----+
|issuer_precinct|violation_code|count|
+---------------+--------------+-----+
|             18|            14|50150|
|             19|            46|48445|
|             14|            14|45036|
|              1|            14|38354|
|             19|            38|36386|
+---------------+--------------+-----+
only showing top 5 rows



In [53]:
top_precinct=parking_violation_2017.filter('issuer_precinct !=0').groupby(col('issuer_precinct')).\
count().withColumnRenamed("count","violation_freq").\
sort(col('violation_freq').desc()).select('issuer_precinct','violation_freq').limit(3)

In [54]:
top_precinct.show()

+---------------+--------------+
|issuer_precinct|violation_freq|
+---------------+--------------+
|             19|        266961|
|             14|        200495|
|              1|        168740|
+---------------+--------------+



In [55]:
df=top_precinct.join(precinct_viol_count,top_precinct.issuer_precinct==precinct_viol_count.issuer_precinct,"inner").\
select(top_precinct.issuer_precinct,'violation_code','count',col('violation_freq').alias('total_violation_freq'))

In [56]:
window = Window.partitionBy(df['issuer_precinct']).orderBy(df['count'].desc())

df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 3).\
sort(col('total_violation_freq').desc(),col('count').desc())\
.select('issuer_precinct','violation_code','count').show() 

+---------------+--------------+-----+
|issuer_precinct|violation_code|count|
+---------------+--------------+-----+
|             19|            46|48445|
|             19|            38|36386|
|             19|            37|36056|
|             14|            14|45036|
|             14|            69|30464|
|             14|            31|22555|
|              1|            14|38354|
|              1|            16|19081|
|              1|            20|15408|
+---------------+--------------+-----+



In [57]:
from pyspark.sql import functions as f
w = Window.partitionBy('issuer_precinct')
ranking=Window.partitionBy('issuer_precinct')
precinct_viol_total_count= precinct_viol_count.select('issuer_precinct','violation_code','count',\
                                                     f.sum('count').over(w).alias('total_freq'))
#                                                       .\
#                                                      rank().over(ranking).alias('rank'))
# precinct_viol_count.withColumn('total_freq',\
# precinct_viol_count.groupBy(precinct_viol_count.issuer_precinct).sum(precinct_viol_count.count).\
# over(Window.partitionBy(precinct_viol_count.issuer_precinct)))



# w = Window.partitionBy('x')
# df.select('x', 'y', f.count('x').over(w).alias('n')).sort('x', 'y').show()


In [58]:
precinct_viol_total_count.show(5)


+---------------+--------------+-----+----------+
|issuer_precinct|violation_code|count|total_freq|
+---------------+--------------+-----+----------+
|              1|            14|38354|    168740|
|              1|            16|19081|    168740|
|              1|            20|15408|    168740|
|              1|            46|12745|    168740|
|              1|            38| 8535|    168740|
+---------------+--------------+-----+----------+
only showing top 5 rows



In [59]:
ranking_total=Window.partitionBy('issuer_precinct').orderBy(col('total_freq').desc())
ranking_viol=Window.partitionBy('issuer_precinct').orderBy(col('count').desc())

precinct_viol_total_count.select('*', rank().over(ranking_total).alias('ranking_total'),\
                                 f.rank().over(ranking_viol).alias('ranking_viol')).\
filter((col('ranking_total') <= 3) & (col('ranking_viol') <= 3)).sort(col('total_freq').desc(),col('count').desc()).\
select('issuer_precinct','violation_code','count','total_freq','ranking_viol','ranking_total').show(50) 
# sort(col('total_freq').desc())\

+---------------+--------------+-----+----------+------------+-------------+
|issuer_precinct|violation_code|count|total_freq|ranking_viol|ranking_total|
+---------------+--------------+-----+----------+------------+-------------+
|             19|            46|48445|    266961|           1|            1|
|             19|            38|36386|    266961|           2|            1|
|             19|            37|36056|    266961|           3|            1|
|             14|            14|45036|    200495|           1|            1|
|             14|            69|30464|    200495|           2|            1|
|             14|            31|22555|    200495|           3|            1|
|              1|            14|38354|    168740|           1|            1|
|              1|            16|19081|    168740|           2|            1|
|              1|            20|15408|    168740|           3|            1|
|             18|            14|50150|    162994|           1|            1|

In [60]:
ranking_total=Window.orderBy(col('total_freq').desc())
ranking_viol=Window.partitionBy('issuer_precinct').orderBy(col('count').desc())

precinct_viol_total_count.select('*',f.dense_rank().over(ranking_total).alias ('ranking_total'),\
                                  f.rank().over(ranking_viol).alias('ranking_viol')).\
filter((col('ranking_total') <= 3) & (col('ranking_viol') <= 3)).sort(col('total_freq').desc(),col('count').desc()).\
select('issuer_precinct','violation_code','count','total_freq','ranking_viol','ranking_total').show(50) 
# sort(col('total_freq').desc())\

+---------------+--------------+-----+----------+------------+-------------+
|issuer_precinct|violation_code|count|total_freq|ranking_viol|ranking_total|
+---------------+--------------+-----+----------+------------+-------------+
|             19|            46|48445|    266961|           1|            1|
|             19|            38|36386|    266961|           2|            1|
|             19|            37|36056|    266961|           3|            1|
|             14|            14|45036|    200495|           1|            2|
|             14|            69|30464|    200495|           2|            2|
|             14|            31|22555|    200495|           3|            2|
|              1|            14|38354|    168740|           1|            3|
|              1|            16|19081|    168740|           2|            3|
|              1|            20|15408|    168740|           3|            3|
+---------------+--------------+-----+----------+------------+-------------+

In [61]:
ranking_total=Window.partitionBy(col('issuer_precinct')).orderBy(col('total_freq').desc())
# ranking_viol=Window.partitionBy('issuer_precinct').orderBy(col('count').desc())

precinct_viol_total_count.select('*', f.rank().over(ranking_total).alias('ranking_total')).\
select('*').show()

+---------------+--------------+-----+----------+-------------+
|issuer_precinct|violation_code|count|total_freq|ranking_total|
+---------------+--------------+-----+----------+-------------+
|              1|            14|38354|    168740|            1|
|              1|            16|19081|    168740|            1|
|              1|            20|15408|    168740|            1|
|              1|            46|12745|    168740|            1|
|              1|            38| 8535|    168740|            1|
|              1|            17| 7526|    168740|            1|
|              1|            37| 6470|    168740|            1|
|              1|            31| 5853|    168740|            1|
|              1|            69| 5672|    168740|            1|
|              1|            19| 5375|    168740|            1|
|              1|            10| 4712|    168740|            1|
|              1|            40| 4592|    168740|            1|
|              1|            21| 4055|  

In [62]:
precinct_viol_total_count.tail(20)

[Row(issuer_precinct=985, violation_code=50, count=8, total_freq=734),
 Row(issuer_precinct=985, violation_code=70, count=8, total_freq=734),
 Row(issuer_precinct=985, violation_code=51, count=6, total_freq=734),
 Row(issuer_precinct=985, violation_code=99, count=5, total_freq=734),
 Row(issuer_precinct=985, violation_code=46, count=5, total_freq=734),
 Row(issuer_precinct=985, violation_code=74, count=3, total_freq=734),
 Row(issuer_precinct=985, violation_code=78, count=3, total_freq=734),
 Row(issuer_precinct=985, violation_code=60, count=2, total_freq=734),
 Row(issuer_precinct=985, violation_code=67, count=1, total_freq=734),
 Row(issuer_precinct=985, violation_code=41, count=1, total_freq=734),
 Row(issuer_precinct=985, violation_code=75, count=1, total_freq=734),
 Row(issuer_precinct=986, violation_code=67, count=1, total_freq=2),
 Row(issuer_precinct=986, violation_code=71, count=1, total_freq=2),
 Row(issuer_precinct=988, violation_code=70, count=6, total_freq=12),
 Row(issuer